# Анализ взаимодействия пользователей с карточками статей на Яндекс.Дзен

---

<p align='center'>
  <img src='https://static.perfluence.net/blog/16243645612588.png' width=800 height=300 />
</p>

---

Каждую карточку в Яндекс.Дзен определяют её тема и источник (у него тоже есть тема). Примеры тем: *«Красота и здоровье»*, *«Россия»*, *«Путешествия»*. Пользователей системы характеризует возрастная категория. К примеру, *«26-30»* или *«45+»*.

Есть три способа взаимодействия пользователей с системой:
* Карточка отображена для пользователя (**`show`**)
* Пользователь кликнул на карточку (**`click`**)
* Пользователь просмотрел статью карточки (**`view`**)

Вопросы менеджеров:
* Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
* Как много карточек генерируют источники с разными темами?
* Как соотносятся темы карточек и темы источников?

Предстоит автоматизация процесса – необходимо сделать дашборд.
Дашборд будет основываться на пайплайне, который будет брать данные из таблицы, в которой хранятся сырые данные, трансформировать данные и укладывать их в агрегирующую таблицу. Пайплайн разработан дата-инженерами

---

### Техническое задание

1. Бизнес-задача: анализ взаимодействия пользователей с карточками Яндекс.Дзен <br/> <br/>
2. Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю <br/> <br/>
3. Кто будет основным пользователем дашборда: менеджеры по анализу контента <br/> <br/>
4. Состав данных для дашборда: <br/>
    * История событий по темам карточек (два графика – абсолютные числа и процентное соотношение)
    * Разбивка событий по темам источников
    * Таблица соответствия тем источников темам карточек <br/> <br/>
5. По каким параметрам данные должны группироваться: <br/>
    * Дата и время
    * Тема карточки
    * Тема источника
    * Возрастная группа <br/> <br/>
6. Характер данных: <br/>
    * История событий по темам карточек — абсолютные величины с разбивкой по минутам
    * Разбивка событий по темам источников — относительные величины (% событий)
    * Соответствия тем источников темам карточек – абсолютные величины <br/> <br/>
7. Важность: все графики имеют равную важность <br/> <br/>
8. Источники данных для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица `log_raw`), для реализации дашборда подготовлен срез данных в таблице **`dash_visits`** <br/> <br/>
8. База данных, в которой будут храниться агрегированные данные: дополнительные агрегированные таблицы в БД `data-analyst-zen-project-db` <br/> <br/>
9. Частота обновления данных: один раз в сутки, в полночь по UTC

### Утвержденный макет дашборда

<p align='center'>
  <img src='https://pictures.s3.yandex.net/resources/Untitled_-_2020-07-06T160925.436_1594041010.png' width=700 height=400 />
</p>

## Содержание проекта
---
1. [Получение данных из БД](#Получение-данных-из-БД) <br/>
    1.1 [Вывод](#Вывод)
2. [Дашборд для менеджеров](#Дашборд-для-менеджеров)
3. [Презентация с ответами на вопросы](#Презентация-с-ответами-на-вопросы)
4. [Итоговый вывод](#Итоговый-вывод)

## Получение данных из БД
---

Импортируем необходимые библиотеки:

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display

Зададим параметры подключения к БД с помощью словаря `db_config` и сформируем с их помощью строки соединения `connection_string`:

In [2]:
# задание параметров подключения к БД
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

# формирование строки соединения с БД
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

Создадим новый экземпляр класса `sqlalchemy`, который предоставляет подключение к базе данных и сохраним его в переменную `engine`:

In [3]:
# подключение к БД
engine = create_engine(connection_string) 

В переменную `query` сформируем и запишем текст запроса PostgreSQL: дословно – *«выбрать все атрибуты из отношения `dash_visits`»*:

In [4]:
# формирование sql-запроса
query = ''' SELECT *
            FROM dash_visits
        '''

Выполним запрос, результирующий набор сохраним в переменной `dash_visits`:

In [5]:
# выполнение запроса с сохранением результата выполнения в таблицу dash_visits
dash_visits = pd.io.sql.read_sql(query, con=engine) 

Выведем первые пять строк датафрейма `dash_visits` и общую информацию о нем:

In [6]:
# получение первых пяти строк датафрейма dash_visits с помощью
# метода head() и общей информации методом info()
display(dash_visits.head())
dash_visits.info()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
record_id       30745 non-null int64
item_topic      30745 non-null object
source_topic    30745 non-null object
age_segment     30745 non-null object
dt              30745 non-null datetime64[ns]
visits          30745 non-null int64
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


Получено `30 745` записей в типами данных – целочисленный `int64`, строковый объект `object` и значения даты и времени `datetime64`.
Опишем содержание столбцов:
* **`record_id`** – id записи
* **`item_topic`** – тема карточки
* **`source_topic`** – тема источника
* **`age_segment`** – возрастная категория
* **`dt`** – время визита 
* **`visits`** – количество визитов

Значений `NaN` не наблюдается, названия столбцов и типы данных в них валидны. 

Для последующей работы с Tableau сохраним датафрейм `dash_visits` в файл ***dash_visits.csv***:

In [7]:
# сохранение датафрейма dash_visits в файл с помощью метода to_csv()
dash_visits.to_csv('dash_visits.csv')

### Вывод
---

На данном этапе было произведено подключение к базе данных посредством библиотеки `sqlalchemy` и выполнен SQL-запрос. Полученная выборка сохранена в таблицу `dash_visits`, которая содержит информацию о количестве и времени визитов пользователей Яндекс.Дзен с указанием их возрастной категории, а также тем источников и карточек, просмотренных ими. В заключении таблица `dash_visits` была экспортирована в файл *dash_visits.csv*

---

## Дашборд для менеджеров
---

Дашборд в облаке [*Tableau Public*](https://public.tableau.com/app/profile/nikita.girya/viz/dashboard_zen/sheet4)

## Презентация с ответами на вопросы
---

Презентация на [*Google.Диске*](https://drive.google.com/file/d/1N1HYPzVGkVH1B0q_TIyjXlOZOEQ0WJZE/view?usp=sharing)

## Итоговый вывод
---

На первом этапе было произведено подключение к базе данных посредством библиотеки `sqlalchemy` и выполнен SQL-запрос. Полученная выборка сохранена в таблицу `dash_visits`, которая содержит информацию о количестве и времени визитов пользователей Яндекс.Дзен с указанием их возрастной категории, а также тем источников и карточек, просмотренных ими. В заключении этапа таблица `dash_visits` была экспортирована в файл *dash_visits.csv* <br/> <br/>

Далее был составлен согласно утвержденному макету и размещен в облаке *Tableau Public* дашборд для менеджеров по анализу контента. Также подготовлена и размещена на *Google.Диске* презентация в формате *.pdf* с ответами на поставленные менеджерами вопросы. По итогу имеем следующие выводы:

* Имеем **около 60 тыс. событий в пике** взаимодействий <br/> Лидеры по количеству визитов – темы карточек **История**, **Наука**, **Общество**, **Отношения**, **Подборки** и **Полезные советы** (в среднем 3-4 тыс.). Остальные категории – 1,5-2,5 тыс. событий <br/> <br/>

* Больше всего карточек генерируют темы **Семейные отношения** (10,7%), **Россия** (9,6%), **Полезные советы** (8,8%), **Путешествия** (7,8%), **Знаменитости** (7,7%) и **Кино** (6,5%) <br/> <br/>

* Самое большое соотношение тема карточки – тема источника – **Рассказы о путешествиях** (4 587), далее **Россия и общество** (3 471), на третьем месте **Кино и наука** (3 279)

---
[**Вернуться к началу**](#Анализ-взаимодействия-пользователей-с-карточками-статей-на-Яндекс.Дзен)